In [2]:
%load_ext cuml.accel
%load_ext cudf.pandas

ModuleNotFoundError: No module named 'cuml.accel'

In [3]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier



In [4]:
df = pd.read_csv('./data/wine_data.csv')
df

,Alcohol,Malic,Ash,Alcalinity,Magesium,Phenols,Flavanoids,Nonflavanoids,Proanthocyanins,Color,Hue,Dilution,Proline,class
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065,0
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050,0
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185,0
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480,0
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740,2
174,13.40,3.91,2.48,23.0,102,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750,2
175,13.27,4.28,2.26,20.0,120,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835,2
176,13.17,2.59,2.37,20.0,120,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840,2


In [6]:
features = ['Alcohol',	'Malic',	'Ash',	'Alcalinity',	'Magesium',	'Phenols',	'Flavanoids',	'Nonflavanoids',	'Proanthocyanins',	'Color',	'Hue',	'Dilution',	'Proline']
X= df[features]
y = df['class']


In [7]:
from sklearn.model_selection import train_test_split
X_tn, X_te, y_tn, y_te = train_test_split(X, y, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
std_scale = StandardScaler()
std_scale.fit(X_tn)
X_tn_std = std_scale.transform(X_tn)
X_te_std = std_scale.transform(X_te)

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
clf_gbt = GradientBoostingClassifier(max_depth=2, learning_rate=0.1, random_state=0)
clf_gbt.fit(X_tn_std, y_tn)

GradientBoostingClassifier(max_depth=2, random_state=0)

In [13]:
pred_gboost =clf_gbt.predict(X_te_std)
pred_gboost

array([0, 2, 1, 0, 1, 0, 0, 2, 1, 1, 2, 2, 0, 1, 2, 1, 0, 0, 2, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 1, 1, 2, 0, 0, 1, 1,
       1])

In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te, pred_gboost)

0.9555555555555556

In [25]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor

clf1 = SVR(kernel='linear')
clf2 = DecisionTreeRegressor()

clf_stack = StackingRegressor(
    estimators=[
        ('svr', clf1),
        ('tree', clf2)],
    final_estimator=LinearRegression())
clf_stack.fit(X_tn_std, y_tn)

StackingRegressor(estimators=[('svr', SVR(kernel='linear')),
                              ('tree', DecisionTreeRegressor())],
                  final_estimator=LinearRegression())

In [28]:
stack_pred = clf_stack.predict(X_te_std)
stack_pred

array([ 1.48859282e-01,  2.08916582e+00,  9.18543233e-01,  3.95328392e-01,
        1.00929720e+00,  1.40692266e-01, -2.82870965e-02,  2.14762703e+00,
        8.40878352e-01,  1.10638031e+00,  1.62831479e+00,  2.06303000e+00,
       -1.42059208e-01,  5.80075785e-01,  1.88813638e+00,  9.42634824e-01,
        5.03033347e-02, -3.64302087e-01,  1.62005429e+00, -9.44680799e-02,
        6.26872814e-01,  3.51964668e-01,  6.73191576e-01,  8.18729379e-01,
        1.17528250e+00,  1.09383563e+00,  8.39221990e-01,  1.14107647e+00,
        8.32266070e-01,  1.99587004e+00, -1.80184605e-03,  2.09087116e-01,
        9.91750102e-01,  2.52304095e-01,  3.79695341e-02,  2.82718480e-01,
        1.76789808e+00,  1.14029236e+00,  1.16267309e+00,  1.97598756e+00,
        1.53408639e-01,  1.26117748e-01,  9.59674408e-01,  1.05226285e+00,
        1.19413864e+00])

In [27]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_te, pred_stack)

0.06380364355842144

In [31]:
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

param_grid = { 'kernel' : ('linear', 'rbf'), 'C': [0.5, 1, 10 , 100]}
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
svc = svm.SVC(random_state=0)
grid_cv = GridSearchCV(svc, param_grid, cv=kfold, scoring='accuracy')
grid_cv.fit(X_tn_std, y_tn)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
             estimator=SVC(random_state=0),
             param_grid={'C': [0.5, 1, 10, 100], 'kernel': ('linear', 'rbf')},
             scoring='accuracy')

In [32]:
grid_cv.cv_results_

{'mean_fit_time': array([0.00203819, 0.00234232, 0.00150013, 0.00122519, 0.00100741,
        0.00116463, 0.00099988, 0.00116124]),
 'std_fit_time': array([1.53102113e-04, 1.31977982e-05, 4.28484554e-04, 4.30838667e-05,
        1.68186353e-05, 1.34256548e-05, 1.72819101e-05, 1.31226418e-05]),
 'mean_score_time': array([0.00122781, 0.00125999, 0.00079079, 0.00063491, 0.00057096,
        0.00061474, 0.00056982, 0.00061355]),
 'std_score_time': array([1.36665922e-04, 2.56715593e-05, 2.38215045e-04, 1.13693419e-05,
        4.60684913e-06, 3.46355816e-06, 3.46158818e-06, 4.79594840e-06]),
 'param_C': masked_array(data=[0.5, 0.5, 1.0, 1.0, 10.0, 10.0, 100.0, 100.0],
              mask=[False, False, False, False, False, False, False, False],
        fill_value=1e+20),
 'param_kernel': masked_array(data=['linear', 'rbf', 'linear', 'rbf', 'linear', 'rbf',
                    'linear', 'rbf'],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
  